In [1]:
# get drunkposture data and pick out posture data that can identify wet plates
# Conny Lin | May 19, 2020

In [18]:
# define local variables
# dataoutput dir
dir_save = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'
# test file path
pfile_test = '/Volumes/COBOLT/MWT DISCARD/20140707X_NG_100s30x10s10s_npr_wetplates/N2/20140709_173751/Rb_5x3_t96h20C_100s30x10s10s_C0109ad.drunkposture2.dat'

In [19]:
import os
import pickle
from pathlib import Path
from pathlib import PurePath
import numpy as np
import pandas as pd

Note: ptyhon3 updated map to iterable. To convert map object to np, needs to use `np.fromiter`:
https://stackoverflow.com/questions/28524378/convert-map-object-to-numpy-array-in-python-3

In [6]:
# look at worm behavior - drunkpoasture2
# test file -----------------------
# the first row of data is loaded as header, but the first row of data usually are very noisy and never used for analysis. So it's ok to lose the data.
df = pd.read_csv(pfile_test,delim_whitespace=True)
# load column names
filename = os.path.join(dir_save,'legend_drunkposture2.pickle')
with open(filename,'rb') as f:
    column_names = pickle.load(f)
# get files that are between 90-95s
i = (df.iloc[:,0] < 95) & (df.iloc[:,0] > 90)
df_short = df.loc[i,:].copy()
df_short.columns = column_names['name']
# get the mean values of all the measures between 90-95s
mean_data = df_short.mean()
mean_data

In [54]:
# load all paths pickle
filename = os.path.join(dir_save,'path_drunkposture.pickle')
with open(filename,'rb') as f:
    filepaths = pickle.load(f)
# check number of files
print(f'{len(filepaths)} drunkposture2.dat files found')
# load column names
filename = os.path.join(dir_save,'legend_drunkposture2.pickle')
with open(filename,'rb') as f:
    column_names = pickle.load(f)
# initialize output array
data = np.empty((len(filepaths),len(column_names)))
data[:] = np.NaN
# initiate iteration counter
itern=0
# combine multiple means - # get mean values from 90-95s recording
for pfile, row_number in zip(filepaths,range(0,data.shape[0])):
    #iteration reporter
    itern+=1
    if (itern%100==0):
        print(f'iterating {itern}th files')
    # the first row of data is loaded as header, but the first row of data usually are very noisy and never used for analysis. So it's ok to lose the data.
    df = pd.read_csv(pfile,delim_whitespace=True)
     # get files that are between 90-95s
    i = (df.iloc[:,0] < 95) & (df.iloc[:,0] > 90)
    df_short = df.loc[i,:].copy()
    df_short.columns = column_names['name']
    # declare var
    mean_data = []
    # get the mean values of all the measures between 90-95s
    mean_data = df_short.mean().values
    # check data for integrity
    if len(mean_data)==len(column_names):
        data[row_number,:] = mean_data # put in data
    else:
        display(mean_data)
        assert False,'something wrong with data'
print('complete')

iterating 100th files
iterating 200th files
iterating 300th files
iterating 400th files
iterating 500th files
iterating 600th files
iterating 700th files
iterating 800th files
iterating 900th files
iterating 1000th files
iterating 1100th files
iterating 1200th files
iterating 1300th files
iterating 1400th files
iterating 1500th files
iterating 1600th files
iterating 1700th files
iterating 1800th files
iterating 1900th files
iterating 2000th files
iterating 2100th files
iterating 2200th files
iterating 2300th files
iterating 2400th files
iterating 2500th files
iterating 2600th files
iterating 2700th files
iterating 2800th files
iterating 2900th files
iterating 3000th files
iterating 3100th files
iterating 3200th files
iterating 3300th files
iterating 3400th files
iterating 3500th files
iterating 3600th files
iterating 3700th files
iterating 3800th files
iterating 3900th files
iterating 4000th files
iterating 4100th files
iterating 4200th files
iterating 4300th files
iterating 4400th fil

array([0.32768754, 0.30255009, 0.25618845, ..., 0.14711226, 0.17202842,
       0.13170652])

In [74]:
# create dataframe
# get paths in dataframe as identifyer
df = pd.concat([pd.DataFrame(filepaths,columns=['path']),
                pd.DataFrame(data,columns=column_names['name'].values)], 
               axis=1)

In [100]:
# report on size
import sys
print(f'df is {(sys.getsizeof(df)/1000000):.2f} MB')

df is 1.46 MB


In [93]:
# QA nan values
print(f'any nan values? {any(df.isna())}')
display(df.describe())
df.info()

any nan values? True


,time,number,goodnumber,speed,length,width,aspect,kink,bias,curve,area,midline,morphwidth
count,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000
mean,92.499563,59.542154,35.134842,0.160702,0.952386,0.236346,0.261861,41.661620,0.644839,27.045645,0.146036,1.061386,0.113863
std,0.061917,48.098739,21.275909,0.063495,0.212908,0.061150,0.074656,11.400845,0.212876,7.164223,0.043805,0.232854,0.026616
min,90.341556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,92.484848,28.568736,19.537520,0.119152,0.860669,0.198795,0.217909,35.264579,0.539718,22.439959,0.123282,0.965725,0.102239
50%,92.500118,49.096014,32.953114,0.157478,0.984346,0.239599,0.265571,41.760140,0.687047,28.968316,0.147437,1.095719,0.114758
75%,92.514787,76.321654,47.234365,0.196934,1.078586,0.276874,0.306581,48.102380,0.795695,31.930484,0.171427,1.196575,0.127020
max,92.878854,736.100000,140.111111,0.493735,1.708762,0.520229,0.625270,125.221212,1.000000,49.041791,0.427343,1.911568,0.278553


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4895 entries, 0 to 4894
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   path        4895 non-null   object 
 1   time        4862 non-null   float64
 2   number      4862 non-null   float64
 3   goodnumber  4862 non-null   float64
 4   speed       4862 non-null   float64
 5   length      4862 non-null   float64
 6   width       4862 non-null   float64
 7   aspect      4862 non-null   float64
 8   kink        4862 non-null   float64
 9   bias        4862 non-null   float64
 10  curve       4862 non-null   float64
 11  area        4862 non-null   float64
 12  midline     4862 non-null   float64
 13  morphwidth  4862 non-null   float64
dtypes: float64(13), object(1)
memory usage: 535.5+ KB


In [101]:
# need to save this to csv
df.to_csv(os.path.join(dir_save, 'drunkposture2_90s_95s_mean.csv'))